За год гитхаб nomeroff-net сильно изменился и код neural_network_deprecated.ipynb перестал работать. Nikolay Shcherbakov обновил код и ноутбук снова заработал, за что Николаю огромное спасибо.

In [ ]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from typing import Callable, Iterable, Dict, List
from tqdm.notebook import tqdm
from concurrent.futures import ProcessPoolExecutor

# Ставим либу с нейронкой

Мы будем использовать open-source проект [nomeroff-net](https://github.com/ria-com/nomeroff-net) для распознавания номеров, а точнее говоря, его форк с исправленными импортами

In [ ]:
!git clone https://github.com/Kucev/nomeroff-net.git


При запуске в colab может написать ошибку установки пакетов:
```Python
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.
```

Но на самом деле - [`usually these error messages are nothing to worry about`](https://github.com/apple/turicreate/issues/3383) (аналогичная проблема возникла у ankitthakur007 работая в colab). 

Можно не переживать - все будет корректно работать, при желании можно перезапустить cell и ошибка исчезнет.

In [ ]:
! pip install -r nomeroff-net/requirements.txt

# Скачиваем изображения

In [ ]:
# У каждого студента свой вариант
VARIANT = 100
VARIANT_URL = f"https://storage.yandexcloud.net/plates/tasks_spring_2021/{VARIANT}.csv"

# Получаем датафрейм с ссылками на изображения
images_links = pd.read_csv(VARIANT_URL)
images_links.head(3)

# Создаем папку для хранения данных
os.makedirs('images', exist_ok = True)

In [ ]:
# Параллельно загружаем изображения
def download_image(row: Dict[str, str]):
    bin_image = requests.get(row['url']).content
    return (row['id'], bin_image)

def parallel_download(load_func: Callable, iterable: List[Dict[str, str]]):
    with ProcessPoolExecutor(max_workers=16) as executor:
        images = list(tqdm(executor.map(load_func, iterable), total=len(iterable)))
    return images

images = parallel_download(download_image, images_links.to_dict('records'))

In [ ]:
# Последовательно пишем на диск
for filename, bin_image in tqdm(images):
    with open(f"images/{filename}", 'wb') as file:
        file.write(bin_image)

# Прогоняем скачанные изображения через нейронную сеть

Рекомендуется запускать этот ноутбук в Google Colab

In [ ]:
# нейронку можно запустить на GPU, либо на CPU
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

In [ ]:
import cv2
import sys

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('nomeroff-net/')
sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.YoloV5Detector import Detector
from NomeroffNet.BBoxNpPoints import NpPointsCraft, convertCvZonesRGBtoBGR, getCvZoneRGB, reshapePoints
from NomeroffNet.OptionsDetector import OptionsDetector
from NomeroffNet.TextDetectors.ru import ru

In [ ]:
# models loading

detector = Detector()
detector.load();

npPointsCraft = NpPointsCraft()
npPointsCraft.load();

optionsDetector = OptionsDetector()
optionsDetector.load("latest");

textDetector = ru
textDetector.load("latest");

In [ ]:
import traceback
import warnings
warnings.filterwarnings("ignore")

class EmptyTargetBoxes(Exception):
  """Модель не обнаружила номер на фото"""
  pass

def parse_boxes_detector_out(model_output):
    """
    Из всех распознанных box с номерами вовзращаем только один наибольшей площади,
      (поскольку задача найти номер ближайшего авто), а также саму площадь и accuracy модели
    """
    boxes = [([box], (box[2] - box[0]) * (box[3]-box[1]), box[4]) for box in model_output]
    sort_by_area = sorted(boxes, key=lambda x: x[1], reverse=True)
    if sort_by_area:
      return sort_by_area[0]
    else:
      raise EmptyTargetBoxes()

# Ниже может выпасть исключение на моменте вызова функции parse_boxes_detector_out, а именно sort_by_area - IndexError
# Когда модель не смогла обнаружить какой-либо target_box, предлагается воспользоваться сервисом Толока для их обработки

number_not_founded = []
model_outputs = []
for filename in tqdm(os.listdir("images")):
    try:
        img = cv2.imread(f"images/{filename}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # detected target Boxes with number
        target_boxes = detector.detect_bbox(img)
        target_box, box_square, box_acc = parse_boxes_detector_out(target_boxes)
        
        # Ниже все контейнеры будут иметь одну и то же кол-во элементов (1), 
        # поскольку мы обрабатываем только один крупнейший target Box
        all_points = npPointsCraft.detect(img, target_box,[5,2,0])
        zones = convertCvZonesRGBtoBGR([getCvZoneRGB(img, reshapePoints(rect, 1)) for rect in all_points])

        # определение региона
        region_ids, _ = optionsDetector.predict(zones)
        region = optionsDetector.getRegionLabels(region_ids)[0]

        # финальный результат + выход с фин слоя нейронки
        text, neur_out = textDetector.predict(zones, return_acc=True);

        # accuracy OCR
        text_acc = torch.tensor(neur_out).softmax(2)[0].max(axis=1)[0].mean().item()

        # собираем результат
        img_data = {
            "filename": filename,
            "latin_text": text[0].upper(),
            "text_acc": text_acc,
            "region": region,
            "box_left": int(target_box[0][0]),
            "box_up": int(target_box[0][1]),
            "box_right": int(target_box[0][2]),
            "box_bottom": int(target_box[0][3]),
            "box_square": box_square,
            "box_acc": box_acc
        }
        model_outputs.append(img_data)

    except EmptyTargetBoxes:
        print(f"Номер не найден - {filename}")
        number_not_founded.append(filename)

    except Exception:
        print("-----------------------")
        print(f"Возникло исключение при обработке фото - {filename}")
        print(traceback.format_exc())
        print("-----------------------")

In [ ]:
len(number_not_founded)

### Модель возвращает текст в латинице - переведем его в кириллицу

In [ ]:
latin_to_cyrillic = {
    "a": "а",
    "b": "в",
    "c": "с",
    "e": "е",
    "h": "н",
    "k": "к",
    "m": "м",
    "o": "о",
    "p": "р",
    "t": "т",
    "x": "х",
    "y": "у",
}

In [ ]:
def transform_number(number, mapping):
    transformed = ""
    for ch in number.lower():
        transformed += mapping.get(ch, ch)
    return transformed.upper()

df_from_model = pd.DataFrame(model_outputs)
df_from_model.loc[:, 'cyrillic_text'] = df_from_model.loc[:, 'latin_text'].apply(transform_number, mapping=latin_to_cyrillic)

df_from_model.head(3)

In [ ]:
images_links['predict'] = None
images_links['confidence'] = None
for i in images_links.index:
  id_ = images_links.loc[i,'id']
  if id_ in  list(df_from_model['filename']):
    tmp = df_from_model[df_from_model['filename'] == id_]
    images_links.loc[i,'predict'] = tmp['cyrillic_text'].iloc[0]
    images_links.loc[i,'confidence'] = tmp['box_acc'].iloc[0]

In [ ]:
# сохраняем в ту же директорию результат
images_links.to_csv(f"{VARIANT}_with_prediction_nn.csv", index=None)

# Отрисовка результатов

In [ ]:
  from IPython import display

def drawBoundingBoxes(imageData, latin_text, box_left, box_up, box_right, box_bottom, color):
    """Draw bounding boxes on an image"""
    imgHeight, imgWidth, _ = imageData.shape
    line_size = int((imgHeight + imgWidth) // 900) + 1
    cv2.rectangle(imageData,(box_left, box_up), (box_right, box_bottom), color, line_size)
    cv2.putText(imageData, latin_text, (box_left, box_bottom + 40), 0, 1e-3 * imgHeight, color, line_size)
    return imageData


def show_image(filename, latin_text, text_acc, box_left, box_up, box_right, box_bottom, color=(255, 0, 0), figsize=(12, 12), **__):
    fig, ax = plt.subplots(figsize=figsize)
    img = cv2.imread(f"images/{filename}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = drawBoundingBoxes(img, latin_text, box_left, box_up, box_right, box_bottom, color)
    ax.imshow(img)
    ax.set_title(f"{filename} - {round(text_acc, 3)}")
    display.display(fig)

In [ ]:
show_image(**df_from_model.iloc[12])